# Naive Bayes
We will be implementing the Bernoulli version of Naive Bayes.

## Hyperparameter tweaking
Following the assignment guidelines, we will be tweaking three hyperparameters throughout the testing of the project; m (for the size of the vocabulary), n (for the n most common words in the training data), and k (for the k least common words in the training data), the last 2 of which will not be included in the vocabulary.

In [86]:
m = 2000
n = 200
k = 100

## Fetch data
Fetching the reviews, while taking into account the aforementioned m, n and k hyperparameters.

In [87]:
import tensorflow as tf
import numpy as np

# fetching the reviews
(x_train_raw, y_train), (x_test_raw, y_test) = tf.keras.datasets.imdb.load_data(num_words=m-k, skip_top=n)

# make a dictionary out of the words provided according to the above conditions 
word_index = tf.keras.datasets.imdb.get_word_index()
index2word = dict((i + 3, word) for (word, i) in word_index.items())
index2word[0] = '[pad]'
index2word[1] = '[bos]'
index2word[2] = '[oov]'

# x_train and x_test will consist of strings that represent reviews, but limited to the dictionary provided
x_train_text = np.array([' '.join([index2word[idx] for idx in text]) for text in x_train_raw])
x_test_text = np.array([' '.join([index2word[idx] for idx in text]) for text in x_test_raw])

## Vectorize data

In [89]:
from sklearn.feature_extraction.text import CountVectorizer

binary_vectorizer = CountVectorizer(binary=True)

# transforming the train and test text reviews into binary matrices
x_train_binary = binary_vectorizer.fit_transform(x_train_text)
x_test_binary = binary_vectorizer.transform(x_test_text)

feature_names = binary_vectorizer.get_feature_names_out()

x_train = np.array(x_train_binary.toarray())
x_test = np.array(x_test_binary.toarray())

Vocabulary size: 1670


## Information Gain

In [91]:
import math

def information_gain(class_, feature):
    classes = set(class_)

    Hc = 0
    for c in classes:
        pc = list(class_).count(c)/len(class_)
        Hc += - pc * math.log(pc, 2)
    print('Overall Entropy:', Hc)
    feature_values = set(feature)

    hc_feature = 0
    for feat in feature_values:

        #pf --> P(X=x)
        pf = list(feature).count(feat)/len(feature)
        indices = [i for i in range(len(feature)) if feature[i] == feat]
        classes_of_feat = [class_[i] for i in indices]
        for c in classes:
            #pcf --> P(C=c|X=x)
            pcf = classes_of_feat.count(c)/len(classes_of_feat)
            if pcf != 0:
                # - P(X=x) * P(C=c|X=x) * log2(P(C=c|X=x))
                temp_H = - pf * pcf * math.log(pcf, 2)
                #sum for all values of C (class) and X (values of specific feature)
                hc_feature += temp_H
    ig = Hc - hc_feature
    return ig

## Bernoulli Naive Bayes

In [107]:
class BernoulliNaiveBayes:
    def __init__(self):
        """
        apriori_positive: Initialised to the total number of positive reviews / the total number of reviews from our training data
        apriori_negative: Initialised to the total number of negative reviews / the total number of reviews from our training data
        """
        self.apriori_positive = None
        self.apriori_negative = None
        self.positive_category_feature_exists = None
        self.negative_category_feature_exists = None
    
    def fit(self, x, y):
        alpha = 1   # the parameter used for Laplace smoothing
        
        num_train_data = x.shape[0]
        num_features = x.shape[1]
                
        # apriori_positive/negative: p(C=1)/p(C=0) respectively for the training data
        self.apriori_positive = len(np.where(y == 1)[0]) / len(y)
        self.apriori_negative = len(np.where(y == 0)[0]) / len(y)
        
        """
        positive_category_feature_exists/negative_category_feature_exists are two arrays of size equal to the total amount
        of features (words in the vocabulary), with each index representing the probability the category will be positive
        or negative respectively, given that the feature in that specific index is present in the training data. 
        """        
        self.positive_category_feature_exists = (np.sum(x[y == 1], axis=0) + alpha) / (len(np.where(y == 1)) + 2 * alpha)
        self.negative_category_feature_exists = (np.sum(x[y == 0], axis=0) + alpha) / (len(np.where(y == 0)) + 2 * alpha)     
        
        return self
        

In [106]:
bernoulli_nb = BernoulliNaiveBayes()
bernoulli_nb.fit(x_train, y_train)

12500
Sum: 12500
Length: 12500
False
(1670,)
1670
